# ***QA-VAC NOTEBOOK - Region Selection Plots from DP0***

Author: Gabriel Luan Souza de Oliveira.

Last update: 25/04/2022.

## ***Imported Modules***

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import healpy as hp
import bokeh as bk
import holoviews as hv
import matplotlib.pyplot as plt
import scipy.stats as sst

from matplotlib.pyplot import cm
from astropy.table import Table
from bokeh.io import output_notebook, show
from holoviews import streams, opts
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize

hv.extension('bokeh')
output_notebook()

import copy
from gavodb import DBManager

## ***Loading Data***

### Calling DBManager

In [ ]:
dbm = DBManager()

### Required infos and table names from pid

In [ ]:
main_pid = '6941'
nside = 4096
footprint_area = 301.77 #degrees²
nest = False
limit = False #False for complete tables, integer n for n rows.
verbose = False
table_names_from_pid = dbm.get_tablelist_from_pid(main_pid)
print(f'Tables from PID {main_pid}:',table_names_from_pid)

### Footprint table

In [ ]:
%%time
footprint_table = dbm.table_loading(main_pid,0,columns = None,limit = limit, verbose = verbose)

if 'ra' and 'dec' not in footprint_table.keys():
    ra,dec = hp.pix2ang(nside=nside, ipix=footprint_table['pixel'].astype(int), lonlat = True, nest = nest)
    ra[ra>180] -= 360
    footprint_table['ra'] = ra
    footprint_table['dec'] = dec
    print("'ra' and dec' columns add from healpy pixels.")
    
else:
    print("No columns added.")

In [ ]:
footprint_table

### Full catalog

In [ ]:
%%time
#catalog = dbm.table_loading(main_pid,1,columns = ['ra','dec','coadd_objects_id','z_best'],limit = limit, verbose = verbose,
#                            footprint_table_name = table_names_from_pid[0], sorting_columns = ['coadd_objects_id','pixel'])
#catalog = dbm.table_loading(main_pid,1,columns = ['ra','dec','coadd_objects_id','z_best'],limit = 500000, verbose = verbose)
#ra_dec_catalog = dbm.table_loading(main_pid,1,columns = ['ra','dec'],limit = False, verbose = verbose)
simple_catalog = catalog = dbm.table_loading(main_pid,1,columns = ['ra','dec','coadd_objects_id','z_best'],limit = 5000, verbose = verbose)

In [ ]:
simple_catalog

## ***Plots***

### Plot Functions

In [ ]:
%reload_ext autoreload
def mask_func(signal,mask_dict):
    if mask_dict != None and signal != None:
        assert mask_dict['relation'] in ('less','greater','equal'), 'Input key is not a valid string.'
        #assert type(signal).__name__ == 'ndarray', 'signal must be an numpy array (ndarray)'

        if mask_dict['relation'] == 'less':
            mask = signal < mask_dict['value']

        elif mask_dict['relation'] == 'greater':
            mask = signal > mask_dict['value']

        elif mask_dict['relation'] == 'equal':
            mask = signal == mask_dict['value']
    
    if mask_dict == None and signal != None:
         mask = signal == signal
    
    if mask_dict == None and signal == None:
         mask = slice(0, None, None)
    return mask

def plot_map(ra,dec,signal, verbose = False, map_type = 'scatter',**kwargs):

    ###-------------------------------------------###
    #                Key informations               #
    ###-------------------------------------------###
    
    nside = 4096
    steradian_in_degrees = (180./np.pi)**2
    sky_area = 4*np.pi*steradian_in_degrees   #Area of an sphere measured in degrees^2. Read more on https://www.mathsisfun.com/geometry/steradian.html
    npixels = hp.nside2npix(nside)            #Númeto de pixeis em um Healpix pixelization scheme => 12.0*(nside)**2.0
    area_of_each_pixel = sky_area/(npixels)   #In degrees^2.
    pix_area_arcmin = 3600*area_of_each_pixel #Pixel area in (minutes of degree)².
    
    if verbose == True:
        print('Number of pixels:',npixels,
              '\nSky area:', sky_area,'degrees²',
              '\nArea of each pixel:', area_of_each_pixel, 'degrees²;', pix_area_arcmin,'min^2')
    
    ###-------------------------------------------###
    #                     Mask                      #
    ###-------------------------------------------###
    mask = mask_func(signal, kwargs['mask_dict'])
    
    ###-------------------------------------------###
    #                     Plot                      #
    ###-------------------------------------------###
    
    if map_type == 'scatter':
        
        # Fig config
        plt.clf()
        fig, ax = plt.subplots(1, figsize = kwargs['figsize'])
        ax.set_facecolor('#696969')
    
        # To plot
        if signal is not None: 
            sc = ax.scatter(ra[mask], dec[mask], c = signal[mask],
                            vmin = kwargs['vmin'], vmax = kwargs['vmax'], cmap = kwargs['cmap'], 
                            marker = kwargs['marker'], s=kwargs['marker_size'], linewidths=kwargs['marker_linewidths'])
        else:
            sc = ax.scatter(ra[mask], dec[mask], c = None, color = kwargs['color'], 
                            marker = kwargs['marker'], s=kwargs['marker_size'], linewidths=kwargs['marker_linewidths'])
            
        

        # Axis Stuff
        #ax.set_xlim(0.9*np.nanmin(ra[mask]),1.1*np.nanmax(ra[mask]))
        #ax.set_ylim(0.9*np.nanmin(dec[mask]),1.1*np.nanmax(dec[mask]))
        ax.invert_xaxis()

        # Labels
        ax.set_xlabel('R.A. (degrees)', fontsize = 16)
        ax.set_ylabel('DEC. (degrees)', fontsize = 16)
        ax.set_title(kwargs['title'], fontsize = 20, weight='bold')
        ax.tick_params(axis='both', which='major', labelsize=16)

        # Colorbar
        if kwargs['colorbar'] == True:
            cbar = fig.colorbar(sc, ax = [ax], location = 'right', pad = 0)
            cbar.set_label(kwargs['barlabel'], fontsize = 16, labelpad = 12.0)
            cbar.set_ticks(kwargs['setticks'])
            cbar.ax.tick_params(labelsize=16)

        plt.show()
    
    if map_type == 'hexbin':
        # Fig config
        plt.clf()
        fig, ax = plt.subplots(1, figsize = kwargs['figsize'])
        ax.set_facecolor('#696969')
        
        # To plot
        hexbin = ax.hexbin(ra[mask], dec[mask], gridsize=kwargs['nbins'])
        # Axis Stuff
        #ax.set_xlim(0.9*np.nanmin(ra[mask]),1.1*np.nanmax(ra[mask]))
        #ax.set_ylim(0.9*np.nanmin(dec[mask]),1.1*np.nanmax(dec[mask]))
        ax.invert_xaxis()
        
        # Labels
        ax.set_xlabel('R.A. (degrees)', fontsize = 16)
        ax.set_ylabel('DEC. (degrees)', fontsize = 16)
        ax.set_title(kwargs['title'], fontsize = 20, weight='bold')
        ax.tick_params(axis='both', which='major', labelsize=16)

        # Colorbar
        if kwargs['colorbar'] == True:
            cbar = fig.colorbar(hexbin, ax = [ax], location = 'right', pad = 0)
            cbar.set_label(kwargs['barlabel'], fontsize = 16, labelpad = 12.0)
            cbar.set_ticks(kwargs['setticks'])
            cbar.ax.tick_params(labelsize=16)

        plt.show()
        
    if map_type == 'hist2d':
        # Fig config
        plt.clf()
        fig, ax = plt.subplots(1, figsize = kwargs['figsize'])
        ax.set_facecolor('#696969')
        
        # To plot
        hist_2d = ax.hist2d(ra[mask], dec[mask], bins = kwargs['nbins'])
        # Axis Stuff
        #ax.set_xlim(0.9*np.nanmin(ra[mask]),1.1*np.nanmax(ra[mask]))
        #ax.set_ylim(0.9*np.nanmin(dec[mask]),1.1*np.nanmax(dec[mask]))
        ax.invert_xaxis()
        
        # Labels
        ax.set_xlabel('R.A. (degrees)', fontsize = 16)
        ax.set_ylabel('DEC. (degrees)', fontsize = 16)
        ax.set_title(kwargs['title'], fontsize = 20, weight='bold')
        ax.tick_params(axis='both', which='major', labelsize=16)

        # Colorbar
        if kwargs['colorbar'] == True:
            cbar = fig.colorbar(hist_2d[3], ax = [ax], location = 'right', pad = 0)
            cbar.set_label(kwargs['barlabel'], fontsize = 16, labelpad = 12.0)
            cbar.set_ticks(kwargs['setticks'])
            cbar.ax.tick_params(labelsize=16)

        plt.show()
        
    if map_type == 'gaussian_kde':
        # Fig config
        plt.clf()
        fig, ax = plt.subplots(1, figsize = kwargs['figsize'])
        ax.set_facecolor('#696969')
        
        # To plot
        k = sst.gaussian_kde(np.vstack([ra[mask],dec[mask]]))
        xi, yi = np.mgrid[ra[mask].min():ra[mask].max():kwargs['nbins']*1j, dec[mask].min():dec[mask].max():kwargs['nbins']*1j]
        zi = k(np.vstack([xi.flatten(), yi.flatten()]))/kwargs['footprint_area']
        gk = ax.pcolormesh(xi, yi, zi.reshape(xi.shape), vmin=np.min(zi.reshape(xi.shape)), vmax=np.max(zi.reshape(xi.shape)))
        
        # Axis Stuff
        #ax.set_xlim(0.9*np.nanmin(ra[mask]),1.1*np.nanmax(ra[mask]))
        #ax.set_ylim(0.9*np.nanmin(dec[mask]),1.1*np.nanmax(dec[mask]))
        ax.invert_xaxis()
        
        # Labels
        ax.set_xlabel('R.A. (degrees)', fontsize = 16)
        ax.set_ylabel('DEC. (degrees)', fontsize = 16)
        ax.set_title(kwargs['title'], fontsize = 20, weight='bold')
        ax.tick_params(axis='both', which='major', labelsize=16)

        # Colorbar
        if kwargs['colorbar'] == True:
            cbar = fig.colorbar(gk, ax = [ax], location = 'right', pad = 0)
            cbar.set_label(kwargs['barlabel'], fontsize = 16, labelpad = 12.0)
            cbar.set_ticks(kwargs['setticks'])
            cbar.ax.tick_params(labelsize=16)
        
        print('Integral on 2D plane = ',k.integrate_box([-np.inf,-np.inf],[np.inf,np.inf]))
        plt.show()

### Footprint Map

In [ ]:
params = {'title':f"Footprint Map",
          'barlabel':"Signal",
          'vmin':0,
          'vmax':1,
          'cmap':cm.inferno,
          'setticks':[0,1],
          'mask_dict': None,
          'colorbar': True,
          'marker':',',
          'marker_size':0.5,
          'marker_linewidths':0.5,
          'figsize': [16,9]
         }

plot_map(footprint_table['ra'],footprint_table['dec'], footprint_table['signal'],'scatter', **params)
del footprint_table

### Catalog scatter Map

In [ ]:
params = {'title':f"Catalog scatter Map",
          'color':'white',
          'setticks':None,
          'mask_dict': None,
          'colorbar': False,
          'marker':'.',
          'marker_size':1.0,
          'marker_linewidths':0.5,
          'figsize': [16,9]
         }
plot_map(simple_catalog['ra'],simple_catalog['dec'], None,'scatter', **params)

### Catalog Hexbin count Map

In [ ]:
params = {'title':f"Catalog Hexbin Map",
          'barlabel': 'Counts',
          'setticks':None,
          'mask_dict': None,
          'colorbar': True,
          'marker':'.',
          'marker_size':1.0,
          'marker_linewidths':0.5,
          'figsize': [16,9],
          'nbins': 25
         }
plot_map(simple_catalog['ra'],simple_catalog['dec'], None,map_type = 'hexbin', **params)

### Catalog Hist2D count Map

In [ ]:
params = {'title':f"Catalog Hist2D Map",
          'barlabel': 'Counts',
          'setticks':None,
          'mask_dict': None,
          'colorbar': True,
          'marker':'.',
          'marker_size':1.0,
          'marker_linewidths':0.5,
          'figsize': [16,9],
          'nbins': 25
         }
plot_map(simple_catalog['ra'],simple_catalog['dec'], None, map_type = 'hist2d', **params)

### Catalog Gaussian_kde Density Map

In [ ]:
params = {'title':f"Catalog Gaussian_kde Map",
          'barlabel': 'Density ' r'$(Counts/deg^{2})$',
          'setticks':None,
          'mask_dict': None,
          'colorbar': True,
          'marker':'.',
          'marker_size':1.0,
          'marker_linewidths':0.5,
          'figsize': [16,9],
          'nbins': 25,
          'footprint_area': footprint_area
         }
plot_map(simple_catalog['ra'],simple_catalog['dec'], None, map_type = 'gaussian_kde', **params)